In [132]:
import pandas as pd

df = pd.read_csv("gemini_BTCUSD_2021_1min.csv")

In [133]:
df.head()

,Unix Timestamp,Date,Open,High,Low,Close,Volume
0,1.643330e+12,1/28/2022 0:05,37303.25,37320.35,37303.25,37320.35,0.000022
1,1.643330e+12,1/28/2022 0:04,37144.20,37343.74,37144.20,37303.25,4.265912
2,1.643330e+12,1/28/2022 0:03,37173.42,37173.42,37128.97,37144.20,0.664541
3,1.643330e+12,1/28/2022 0:02,37105.20,37194.73,37105.20,37173.42,1.419849
4,1.643330e+12,1/28/2022 0:01,37141.74,37154.38,37105.20,37105.20,4.741223


In [134]:
import numpy as np
#(max'-min')/(max-min)*(value-max)+max'


data = df.to_numpy(na_value=35000)

print(data.size)
print(data[0][0])
max_iter = 562883
for k in range(2, 7):
    max_val = -1
    min_val = 9999999
    for i in range(max_iter):
        val = data[i - 1][k]
        if val > max_val:
            max_val = val
        if val < max_val:
            min_val = val
    for i in range(max_iter):
        data[i][k] = float(1/(max_val - min_val) * (data[i][k] - max_val) + 1)
    print(data[2][k])
    
print(data[2][2])

3940181
1643330000000.0
0.20368409437454882
0.20261833859215816
0.20365150768514062
0.20190717474217346
-0.00136448628179342
0.20368409437454882


In [135]:

data = np.delete(data, 0, axis=1)
data = np.delete(data, 0, axis=1)


In [136]:
data = data[0:10000]

In [137]:
data_c = data[:, 4]
data = np.delete(data, 4, axis=1)

In [138]:
import torch
#print(type(data[2][4]))

print(data[10])
print(data[1][1])
d_t = torch.from_numpy(data.astype(np.float32))
d_c = torch.from_numpy(data_c.astype(np.float32))
d_c.shape

[0.20370311060490143 0.20401434489172432 0.2047887520221241
 0.2040352772959717]
0.20688552798451465


torch.Size([10000])

In [139]:
n_samples = d_t.shape[0]
n_val = int(.2 * n_samples)
shuffled_indices = torch.randperm(n_samples).numpy()
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [140]:
train_d = d_t[train_indices]
train_d_c = d_c[train_indices]
seq_length = 4
val_d = d_t[val_indices]
val_d_c = d_c[val_indices]

In [141]:
from torch.utils.data import Dataset

class SequenceDataset(Dataset):
    def __init__(self, dataframe, target, features, sequence_length=5):
       # self.features = features
        #self.target = target
        self.sequence_length = sequence_length
        self.y = target
        self.X = features

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, i): 
        if i >= self.sequence_length - 1:
            i_start = i - self.sequence_length + 1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i + 1), :]
            x = torch.cat((padding, x), 0)

        return x, self.y[i]

    

In [142]:
i = 27
sequence_length = 600

train_dataset = SequenceDataset(
    train_d,
    target=train_d_c,
    features=train_d,
    sequence_length=sequence_length
)
test_dataset = SequenceDataset(
    val_d,
    target=val_d_c,
    features=val_d,
    sequence_length=sequence_length
)
X, y = train_dataset[i]
print(X)

tensor([[0.1852, 0.1854, 0.1855, 0.1835],
        [0.1852, 0.1854, 0.1855, 0.1835],
        [0.1852, 0.1854, 0.1855, 0.1835],
        ...,
        [0.1766, 0.1769, 0.1776, 0.1766],
        [0.1847, 0.1843, 0.1846, 0.1840],
        [0.1684, 0.1684, 0.1692, 0.1676]])


In [143]:
from torch.utils.data import DataLoader
torch.manual_seed(99)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [162]:
from torch import nn
device = torch.device('cuda:0')
class ShallowRegressionLSTM(nn.Module):
    def __init__(self, num_sensors, hidden_units):
        super().__init__()
        self.num_sensors = num_sensors  # this is the number of features
        self.hidden_units = hidden_units
        self.num_layers = 1

        self.lstm = nn.LSTM(
            input_size=num_sensors,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers
        )

        self.linear = nn.Linear(in_features=self.hidden_units, out_features=1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()

        _, (hn, _) = self.lstm(x, (h0.to(device), c0.to(device)))
        out = self.linear(hn[0]).flatten()  # First dim of Hn is num_layers, which is set to 1 above.

        return out

In [171]:
learning_rate = 5e-5
num_hidden_units = 16

model = ShallowRegressionLSTM(num_sensors=4, hidden_units=num_hidden_units)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [172]:
import torch.nn as nn
import torch.optim as optim
print(torch.cuda.is_available())
import time
device = torch.device('cuda:0')
kwargs = {'num_workers': 1, 'pin_memory': True}
#train_loader = torch.utils.data.DataLoader(transformed_cifar10, batch_size=64, shuffle=True, **kwargs)
#model = nn.Sequential(nn.Linear(4, 512), nn.Tanh(), nn.Linear(512, 256), nn.Tanh(), nn.Linear(256, 1))
model.to(device)
#transformed_cifar10.to(device)
loss_fn = nn.MSELoss()
#learning_rate = 1e-2
#optimizer = optim.SGD(model.parameters(), lr=learning_rate)
n_epochs = 50

start = time.time()

for epoch in range(n_epochs):
    for X, y in train_loader:
        train_d = X.to(device)
        train_d_c = y.to(device)

        outputs = model(train_d)
        loss = loss_fn(outputs, train_d_c)

        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))
end = time.time()
print(end - start)

True
Epoch: 0, Loss: 0.008705
Epoch: 1, Loss: 0.005518
Epoch: 2, Loss: 0.003388
Epoch: 3, Loss: 0.001875
Epoch: 4, Loss: 0.000763
Epoch: 5, Loss: 0.000477
Epoch: 6, Loss: 0.000086
Epoch: 7, Loss: 0.000433
Epoch: 8, Loss: 0.000142
Epoch: 9, Loss: 0.000025
Epoch: 10, Loss: 0.000028
Epoch: 11, Loss: 0.000111
Epoch: 12, Loss: 0.000038
Epoch: 13, Loss: 0.000065
Epoch: 14, Loss: 0.000031
Epoch: 15, Loss: 0.000080
Epoch: 16, Loss: 0.000024
Epoch: 17, Loss: 0.000017
Epoch: 18, Loss: 0.000046
Epoch: 19, Loss: 0.000029
Epoch: 20, Loss: 0.000019
Epoch: 21, Loss: 0.000020
Epoch: 22, Loss: 0.000046
Epoch: 23, Loss: 0.000097
Epoch: 24, Loss: 0.000050
Epoch: 25, Loss: 0.000095
Epoch: 26, Loss: 0.000052
Epoch: 27, Loss: 0.000101
Epoch: 28, Loss: 0.000023
Epoch: 29, Loss: 0.000052
Epoch: 30, Loss: 0.000136
Epoch: 31, Loss: 0.000025
Epoch: 32, Loss: 0.000130
Epoch: 33, Loss: 0.000193
Epoch: 34, Loss: 0.000112
Epoch: 35, Loss: 0.000037
Epoch: 36, Loss: 0.000102
Epoch: 37, Loss: 0.000024
Epoch: 38, Loss: 

In [173]:
model.eval()

correct = 0
total = 0
with torch.no_grad():
     for X, y in test_loader:
        val_d = X.to(device)
        val_d_c = y.to(device)

        outputs = model(val_d)
        print(outputs[1])
        
        print(val_d_c[1])
        print("\n")


tensor(0.0036, device='cuda:0')
tensor(0.0694, device='cuda:0')


tensor(0.0016, device='cuda:0')
tensor(-0.0006, device='cuda:0')


tensor(0.0031, device='cuda:0')
tensor(-0.0020, device='cuda:0')


tensor(0.0004, device='cuda:0')
tensor(0.0115, device='cuda:0')


tensor(0.0012, device='cuda:0')
tensor(0.0017, device='cuda:0')


tensor(-0.0002, device='cuda:0')
tensor(0.0059, device='cuda:0')


tensor(-0.0019, device='cuda:0')
tensor(0.0093, device='cuda:0')


tensor(-0.0011, device='cuda:0')
tensor(-0.0023, device='cuda:0')


tensor(-0.0014, device='cuda:0')
tensor(-0.0023, device='cuda:0')


tensor(-0.0012, device='cuda:0')
tensor(-0.0025, device='cuda:0')


tensor(0.0008, device='cuda:0')
tensor(-0.0013, device='cuda:0')


tensor(0.0019, device='cuda:0')
tensor(-0.0023, device='cuda:0')


tensor(0.0001, device='cuda:0')
tensor(-0.0022, device='cuda:0')


tensor(0.0026, device='cuda:0')
tensor(-0.0009, device='cuda:0')


tensor(-0.0007, device='cuda:0')
tensor(-0.0021, device='cuda: